# Problem 1

## a)

In [1]:
import numpy as np
import hickle as hkl
import matplotlib.pyplot as plt
from jupyterthemes import jtplot

# Refer to ./data_generator.ipynb for generating u and training examples

d = 100 #dimensions

u_mat = hkl.load('u_mat_rev.hkl')

In [2]:
# Checking quasi-orthogonality for u

for i in range(6):
    for j in range(6):
        u_i  = u_mat[:,i]
        u_j  = u_mat[:,j]
        if (i!=j):
            print('Dot product of u',i+1, ',u',j+1,': ', np.dot(u_i,u_j))
    print('\n')

Dot product of u 1 ,u 2 :  -1.0
Dot product of u 1 ,u 3 :  1.0
Dot product of u 1 ,u 4 :  3.0
Dot product of u 1 ,u 5 :  2.0
Dot product of u 1 ,u 6 :  -3.0


Dot product of u 2 ,u 1 :  -1.0
Dot product of u 2 ,u 3 :  -3.0
Dot product of u 2 ,u 4 :  -3.0
Dot product of u 2 ,u 5 :  1.0
Dot product of u 2 ,u 6 :  -3.0


Dot product of u 3 ,u 1 :  1.0
Dot product of u 3 ,u 2 :  -3.0
Dot product of u 3 ,u 4 :  2.0
Dot product of u 3 ,u 5 :  2.0
Dot product of u 3 ,u 6 :  -4.0


Dot product of u 4 ,u 1 :  3.0
Dot product of u 4 ,u 2 :  -3.0
Dot product of u 4 ,u 3 :  2.0
Dot product of u 4 ,u 5 :  1.0
Dot product of u 4 ,u 6 :  1.0


Dot product of u 5 ,u 1 :  2.0
Dot product of u 5 ,u 2 :  1.0
Dot product of u 5 ,u 3 :  2.0
Dot product of u 5 ,u 4 :  1.0
Dot product of u 5 ,u 6 :  1.0


Dot product of u 6 ,u 1 :  -3.0
Dot product of u 6 ,u 2 :  -3.0
Dot product of u 6 ,u 3 :  -4.0
Dot product of u 6 ,u 4 :  1.0
Dot product of u 6 ,u 5 :  1.0




In [3]:
# Component count check
N = 1000  # varies
d = 100 # varies
filenameP= 'p1_data_'
filenameS = '_dim_' + str(d) + '.hkl'
info = hkl.load(filenameP + str(N) + filenameS)
col = np.array([[1],[0],[0]])
ss = sum(info['label'] @ col)
print(ss)
# 331 365 304

[323.]


In [4]:
N = 1000  # varies
d = 100 # varies
filenameP= 'p1_data_'
filenameS = '_dim_' + str(d) + '.hkl'
info = hkl.load(filenameP + str(N) + filenameS)
multi_dim_data = info['data']

# Demeaning of the data
mean_vec = np.mean(multi_dim_data, axis=0)
nomean_data = multi_dim_data - mean_vec
nomean_data = nomean_data  #(*1/N)

u, s, vt = np.linalg.svd(nomean_data,full_matrices=False)  # returns u, s(1d array), v.T
#print(s)    #arrray of singular values


# Energy Calculation
frobenius_norm_sq = sum(s**2)
#print(frobenius_norm_sq)
d0 = 1
var_exp = 0.95
current_energy = 0
while current_energy < var_exp*frobenius_norm_sq:
    current_energy += (s[d0-1]**2)
    d0 += 1

print(d0)  # components required for explaining $var_exp$ % variance

6


d0 (number of dominant singular values) is 6 everytime. This remains constant as N increases

## (b)

In [27]:
# Considering d0 components only
d0 = 6
u_d0 = u[:,:d0]
smat = np.diag(s[:d0])
low_dim_data = u_d0  @ smat


from sklearn.cluster import KMeans

num_clusters = 3
kmeans = KMeans(init = 'k-means++',n_clusters=num_clusters,verbose=0).fit(low_dim_data)


# Converting cluster labels to one-hot encoding
pred_label = kmeans.labels_ 
pred_label_oh = np.zeros((N,num_clusters))
for i in range(len(pred_label)):
    pred_label_oh[i, pred_label[i]] = 1


kmeans.cluster_centers_.shape

(3, 6)

In [6]:
# Category wise count
# Component count check
list_array = [np.array([[1],[0],[0]]),np.array([[0],[1],[0]]),np.array([[0],[0],[1]])]
for comp in list_array:
    col = comp
    ss = sum(info['label'] @ col)
    print(ss)
    
print('\n')

list_array = [np.array([[1],[0],[0]]),np.array([[0],[1],[0]]),np.array([[0],[0],[1]])]
for comp in list_array:
    col = comp
    ss = sum(pred_label_oh @ col)
    print(ss)

# 3 -> 2
# 2 -> 1
# 1 -> 3

[323.]
[341.]
[336.]


[460.]
[199.]
[341.]


In [24]:
# Generating 3xK table
cnt = np.zeros(num_clusters) # components in each predicted cluster for a given $num_clusters$ 
# [319 350 331] # component wise sample_pt count
true_label = info['label']

comp_label = np.array([[1,0,0],
                      [0,1,0],
                      [0,0,1]])

total_comp_cnt = np.array([323,341,336])

for j in range(3):
    cnt = np.zeros(num_clusters)
    for i in range(1000):
        if np.array_equal(np.array([1,0,0]),pred_label_oh[i,:].reshape(num_clusters)) and np.array_equal(comp_label[j],true_label[i,:].reshape(3)):
            cnt[0]+=1
        elif np.array_equal(np.array([0,1,0]),pred_label_oh[i,:].reshape(num_clusters)) and np.array_equal(comp_label[j],true_label[i,:].reshape(3)):
            cnt[1]+=1
        elif np.array_equal(np.array([0,0,1]),pred_label_oh[i,:].reshape(num_clusters)) and np.array_equal(comp_label[j],true_label[i,:].reshape(3)):
            cnt[2]+=1

    print(cnt/total_comp_cnt[j])

[1. 0. 0.]
[0. 1. 0.]
[0.4077381 0.        0.5922619]


# Problem 2

In [16]:
# d0-dimension projection of u_j's
cluster_centers = kmeans.cluster_centers_
u_1, s_1, vt_1 = np.linalg.svd(u_mat.T, full_matrices=False)

d0 = 6
u_1_d0 = u_1[:,:d0]
s_1_mat = np.diag(s_1[:d0])
proj_u = u_1_d0  @ s_1_mat


#smat = np.diag(s_1)
#proj_u = u_1 @ smat

#print(s_1.shape)
#proj_u.shape

(6, 6)

In [55]:
# Extra
inner_prod = np.inner(cluster_center,proj_u)
inner_prod_norm = np.zeros((num_clusters,d0))
for i in range(len(inner_prod)):
    inner_prod_norm[i,:] = (inner_prod[i,:] - np.mean(inner_prod,1)[i] )/np.std(inner_prod,1)[i]
#inner_prod

In [ ]:
# Extra
plt.plot(np.arange(1,7),inner_prod_norm[0,:],color='r',label='C1')
plt.plot(np.arange(1,7),inner_prod_norm[1,:],color='b', label='C2')
plt.plot(np.arange(1,7),inner_prod_norm[2,:],color='k',label='C3')
plt.grid()
plt.legend()
plt.xlabel(r'$U_j$')

In [17]:
# Key Geometrix interpretation
## Distance,angle calculation between each cluster centre and d0 dimensional u vector
import math
for i in range(num_clusters):
    center = cluster_centers[i,:].reshape(1,d0)
    for j in range(6):
        u_vec = proj_u[j,:].reshape(1,d0)
        #u_vec = u_mat[:,j].reshape(1,d)
        dist = np.linalg.norm(center-u_vec)/np.linalg.norm(u_vec)
        
        unit_c = center/np.linalg.norm(center)
        unit_v = u_vec/np.linalg.norm(u_vec)
        dot_prod = np.dot(unit_c,unit_v.T)
        angle = np.arccos(dot_prod)*180 / math.pi
        print('Distance between C',i+1, ' and U_', j+1,' =',dist, 'Angle=',angle)
    
    print('\n')

Distance between C 1  and U_ 1  = 0.9510598320092452 Angle= [[51.65435667]]
Distance between C 1  and U_ 2  = 1.7480592258004817 Angle= [[105.51561223]]
Distance between C 1  and U_ 3  = 1.0192008785253328 Angle= [[55.20296619]]
Distance between C 1  and U_ 4  = 1.0644193394225383 Angle= [[57.95695239]]
Distance between C 1  and U_ 5  = 1.6843865608908835 Angle= [[81.97248779]]
Distance between C 1  and U_ 6  = 1.8819920506675887 Angle= [[110.16550175]]


Distance between C 2  and U_ 1  = 2.1800230247014447 Angle= [[104.36036094]]
Distance between C 2  and U_ 2  = 1.348603255164232 Angle= [[49.98742478]]
Distance between C 2  and U_ 3  = 2.3515330632442963 Angle= [[116.49475465]]
Distance between C 2  and U_ 4  = 2.507468515652374 Angle= [[131.70521151]]
Distance between C 2  and U_ 5  = 2.4175337151357095 Angle= [[89.427055]]
Distance between C 2  and U_ 6  = 2.3249069386537426 Angle= [[102.50824703]]


Distance between C 3  and U_ 1  = 1.9125568216795867 Angle= [[131.01909656]]
Dista

In [88]:
# Results are intuitive when run on multi-dimensional data (100)